In [2]:
cd ..

/home/is/akiyoshi-n/my-project


In [3]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification, split_test_data, load_text_dataset, load_dataset_2class_classification_v2, split_test_data_stratify, preprocess_for_Trainer, load_sequential_emotion_data
from src.my_project.train_v2 import ActClassifier
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import wandb
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [4]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [22]:
import pandas as pd
# DataFrameをExcelファイルに保存
df = pd.read_excel(f"{DATASET_PATH}/lifestory_annotation.xlsx")  # index=Falseでインデックスを保存しない

In [24]:
# 許可されているラベルのリスト（空白を取り除いた状態）
allowed_labels = [
    "1-A.睡眠",
    "1-B.身の回りの用事",
    "1-C-a.食事-飲酒あり",
    "1-C-b.食事-飲酒なし・不明",
    "2-A-a.通勤・通学",
    "2-A-b.その他の移動",
    "2-B.仕事",
    "2-C.学業",
    "2-D.家事",
    "2-E.介護・看護",
    "2-F.育児",
    "3-A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）",
    "3-B.買い物",
    "3-C-a.趣味・娯楽-体動かす",
    "3-C-b.趣味・娯楽-体動かさない",
    "3-D-a.交際-物理的接触",
    "3-D-b.交際-オンライン的接触",
    "3-E.受診・療養",
    "3-F-a.喫煙",
    "3-F-b.その他の活動(事故など)",
    "4.行動なし"
]

In [25]:
# label1~label8のカラム名
label_columns = ['label1', 'label2', 'label3', 'label4', 'label5', 'label6', 'label7', 'label8']

# ラベルが許可されたリストに含まれているかどうかをチェック
def is_valid_row(row):
    for label in label_columns:
        if pd.notna(row[label]) and ''.join(row[label].split()) not in allowed_labels:
            return False
    return True

In [23]:
df.iloc[3]

text      突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに...
label1                        3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）
label2                                                  NaN
label3                                                  NaN
label4                                                  NaN
label5                                                  NaN
label6                                                  NaN
label7                                                  NaN
label8                                                  NaN
Name: 3, dtype: object

In [21]:
# df.iloc[3]には許可されていないラベルが含まれているので，is_valid_row関数を使ってチェック
is_valid_row(df.iloc[3])

False

In [ ]:
3 - A. メディア（テレビ・ラジオ・新聞・雑誌・ネット）
3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）

In [26]:
# 条件に合わない行を削除
df_new = df[df.apply(is_valid_row, axis=1)]

In [27]:
df_new

,text,label1,label2,label3,label4,label5,label6,label7,label8
0,北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっ...,2 - A - b. その他の移動,4. 行動なし,NaN,NaN,NaN,NaN,NaN,NaN
1,買った商品が翌日特価でさらに安くなっていたこと。,3 - B. 買い物,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。,3 - B. 買い物,2 - D. 家事,NaN,NaN,NaN,NaN,NaN,NaN
3,突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに...,3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,外出先で手袋を片方なくし探したが見つからなかった。,2 - A - b. その他の移動,4. 行動なし,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1359,ここのところ楽しいことはなくて、寝るのが楽しみ,1 - A. 睡眠,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1360,ペットとの過ごす何気ない日々がとてもよい。,2 - F. 育児,1 - A. 睡眠,NaN,NaN,NaN,NaN,NaN,NaN
1361,生まれてくる赤ちゃんの洋服などを下見にいったこと。,2 - D. 家事,3 - B. 買い物,NaN,NaN,NaN,NaN,NaN,NaN
1362,高校生の娘が帰省してくると毎日が穏やかで楽しいですね！,2 - F. 育児,3 - D - a. 交際-物理的接触,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 新しいエクセルファイルとして保存
df_new.to_excel(f"{DATASET_PATH}/lifestory_annotation_new.xlsx", index=False)

In [30]:
# df_newには最大でどこまでのcolumnが存在するか
max_column = 0
for i in range(len(df_new)):
    for j in range(len(df_new.columns)):
        if pd.notna(df_new.iloc[i, j]):
            max_column = max(max_column, j)

In [31]:
max_column

5